In [ ]:
FLASK_ADDR_DEFAULT = "perf-insight.lab.eng.pek2.redhat.com:5000"
ENV_YAML = "/root/.perf-insight.yaml"

WORKSPACE = "."
BASE_TESTRUN_YAML = WORKSPACE + "/base.generate_testrun_results.yaml"
TEST_TESTRUN_YAML = WORKSPACE + "/test.generate_testrun_results.yaml"
METADATA_YAML = WORKSPACE + "/generate_2way_metadata.yaml"
BENCHMARK_YAML = WORKSPACE + "/generate_2way_benchmark.yaml"

BASE_DATASTORE = WORKSPACE + "/base.datastore.json"
TEST_DATASTORE = WORKSPACE + "/test.datastore.json"
BASE_METADATA = WORKSPACE + "/base.metadata.json"
TEST_METADATA = WORKSPACE + "/test.metadata.json"

BASE_TESTRUN_RESULT = WORKSPACE + "/base.testrun_result.csv"
TEST_TESTRUN_RESULT = WORKSPACE + "/test.testrun_result.csv"
METADATA = WORKSPACE + "/2way_metadata.csv"
BENCHMARK = WORKSPACE + "/2way_benchmark.csv"
PARAMETERS = WORKSPACE + "/2way_parameters.csv"
STATISTICS = WORKSPACE + "/2way_statistics.json"
SUMMARY = WORKSPACE + "/2way_summary.csv"

SCRIPTPATH = WORKSPACE + "/utils"

import os
import sys
import yaml
import json
import pandas as pd
from IPython.display import display, HTML, Markdown
from datetime import datetime

BASEPATH = os.path.abspath('.')

try:
    with open(ENV_YAML) as f:
        FLASK_ADDR = "http://" + yaml.safe_load(f).get("jupyter").get("flask_server", FLASK_ADDR_DEFAULT)
except:
    print("Cannot get Flask address. Use default value http://" + FLASK_ADDR_DEFAULT)
    FLASK_ADDR = "http://" + FLASK_ADDR_DEFAULT

# Perf-insight - Benchmark Report

In [ ]:
now = datetime.now()
dt_string = "*Generate time: {}*".format(now.strftime("%Y-%m-%d %H:%M:%S"))
display(Markdown(dt_string))

In [ ]:
# Common functions
def bold_font(s):
    return 'font-weight: bold'

In [ ]:
# Generate Base testrun result script
base_gen_result_script = "{}/generate_testrun_results.py --config {} --datastore {} --metadata {} --output {}".format(
SCRIPTPATH, BASE_TESTRUN_YAML, BASE_DATASTORE, BASE_METADATA, BASE_TESTRUN_RESULT)

# Generate Test testrun result script
test_gen_result_script = "{}/generate_testrun_results.py --config {} --datastore {} --metadata {} --output {}".format(
SCRIPTPATH, TEST_TESTRUN_YAML, TEST_DATASTORE, TEST_METADATA, TEST_TESTRUN_RESULT)

# Generate 2way metadata script
gen_metadata_script = "{}/generate_2way_metadata.py --config {} --test {} --base {} --output {}".format(
SCRIPTPATH, METADATA_YAML, TEST_METADATA, BASE_METADATA, METADATA)

# Generate 2way benchmark script
gen_benchmark_script = "{}/generate_2way_benchmark.py --config {} --test {} --base {} --output {}".format(
SCRIPTPATH, BENCHMARK_YAML, TEST_TESTRUN_RESULT, BASE_TESTRUN_RESULT, BENCHMARK)

# Generate 2way parameters script
gen_parameters_script = "{}/generate_2way_parameters.py --benchmark-config {} --output {}".format(
SCRIPTPATH, BENCHMARK_YAML, PARAMETERS)

# Generate 2way statistics script
gen_statistics_script = "{}/generate_2way_statistics.py --benchmark-csv {} --output {}".format(
SCRIPTPATH, BENCHMARK, STATISTICS)

# Generate 2way summary script
gen_summary_script = "{}/generate_2way_summary.py --statistics-json {} --output {}".format(
SCRIPTPATH, STATISTICS, SUMMARY)


# Run scripts parallelly
import multiprocessing
all_processes = (base_gen_result_script, test_gen_result_script, gen_metadata_script, gen_parameters_script)   

def execute(process):                                                             
    os.system(f'python3 {process}') 

process_pool = multiprocessing.Pool(processes = 4)                                                        
process_pool.map(execute, all_processes)


for result in [BASE_TESTRUN_RESULT, TEST_TESTRUN_RESULT, METADATA]:
    assert os.path.exists(result), "Fail to generate {}! Exit.".format(result)

# Generate 2way benchmark
os.system('python3 {}'.format(gen_benchmark_script))
assert os.path.exists(BENCHMARK), "Fail to generate {}! Exit.".format(BENCHMARK)

# Generate 2way statistics and summary
os.system('python3 {}'.format(gen_statistics_script))
assert os.path.exists(STATISTICS), "Fail to generate {}! Exit.".format(STATISTICS)
os.system('python3 {}'.format(gen_summary_script))
assert os.path.exists(SUMMARY), "Fail to generate {}! Exit.".format(SUMMARY)


In [ ]:
# Format Summary
def summary_color_delta(val):
    with open(BENCHMARK_YAML) as f:
        fillna = yaml.safe_load(f).get('benchmark_comparison_generator').get('defaults').get('fillna', "NaN")
    color_dict = {
        "FAIL": 'color: red; background-color: #FFB6C1',
        "PASS": 'color: green; background-color: #F0FFF0',
        fillna: 'color: #D3D3D3',
    }
    if '+' in val:
        color = 'color: green'
    elif '-' in val:
        color = 'color: red'
    else:
        color = color_dict.get(val, 'color: black')
    return color + '; font-weight: bold'


# Hide header and enlarge font-size
styles = [
    dict(selector="th", props=[("font-size", "0%")]),
    dict(selector="", props=[("font-size", "102%")])
]

if os.path.exists(SUMMARY):
    # Read summary and display
    display(Markdown("## Summary"))
    summary_df = pd.read_csv(SUMMARY, keep_default_na=False, index_col=0)
    display(summary_df.style.set_table_styles(styles).applymap(summary_color_delta,subset=["VALUE"]).hide_index())

## Introduction

In [ ]:
def read_json(json_file):
    with open(json_file, 'r') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print("Fail to load {}".format(json_file))
            raise
    return data

base_metadata = read_json(BASE_METADATA)
test_metadata = read_json(TEST_METADATA)
assert base_metadata.get("testrun-type") == test_metadata.get("testrun-type"), "Base and Test type must be the same! Exit."

run_type = base_metadata.get("testrun-type")
base_platform = base_metadata.get("testrun-platform")
test_platform = test_metadata.get("testrun-platform")
base_id = base_metadata.get("testrun-id")
test_id = test_metadata.get("testrun-id")
# Type and platform must not be None
assert run_type is not None, "Type is None! Exit."
assert base_platform is not None, "Base platform is None! Exit."
assert test_platform is not None, "Test platform is None! Exit."

#with open('{}/templates/introduction_{}_{}.md'.format(BASEPATH, base_platform.lower(), run_type), 'r') as f:
#    display(Markdown(f.read()))

with open('{}/testrun_introduction.md'.format(BASEPATH, base_platform.lower(), run_type), 'r') as f:
    display(Markdown(f.read()))
    
    
if base_platform != test_platform:
    with open('{}/introduction_{}_{}.md'.format(BASEPATH, test_platform.lower(), run_type), 'r') as f:
        display(Markdown('\n'+f.read()))

In [ ]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

## Metadata

In [ ]:
%%HTML
* The differences between Test and Base are <b style='color:orange'>highlighted</b>.

In [ ]:
# Format metadata
def highlight_diff(row, cell_format):
    cell_format = cell_format if row['TEST'] != row['BASE'] else ''
    format_row = ['']*(len(row)-2) + [cell_format, cell_format]
    return format_row

def color_diff(row):
    return highlight_diff(row, 'color: orange')

def bold_diff(row):
    return highlight_diff(row, 'font-weight: bold')
    
conf_df = pd.read_csv(METADATA, index_col=0, keep_default_na=False)
#conf_df = conf_df[['NAME', 'TEST', 'BASE']]
display(conf_df.style.applymap(bold_font, subset=['NAME']).apply(color_diff, axis=1).apply(bold_diff, axis=1).hide_index())

## Benchmark Introduction

In [ ]:
# Show benchmark report description
with open('{}/benchmark_description.md'.format(BASEPATH), 'r') as f:
    display(Markdown(f.read()))

## Benchmark Report

**User parameters**

In [ ]:
# Show user parameters
with open(PARAMETERS) as f:
    param_df = pd.read_csv(f, index_col=0, dtype=str, keep_default_na=False)
    
display(param_df.style.applymap(bold_font, subset=['name']).hide_index())

**Detailed Report**

In [ ]:
# Format benchmark report
def benchmark_color_delta(val):
    with open(BENCHMARK_YAML) as f:
        fillna = yaml.safe_load(f).get('benchmark_comparison_generator').get(
            'defaults').get('fillna', "NaN")
    color_dict = {
        "Dramatic Regression": 'color: red; background-color: #FFB6C1',
        "Moderate Regression": 'color: red',
        "Dramatic Improvement": 'color: green; background-color: #F0FFF0',
        "Moderate Improvement": 'color: green',
        "High Variance": 'color: orange; background-color: #FAFAD2',
        "No Significance": 'color: gray',
        "Negligible Changes": 'color: gray',
        "Invalid Data": 'color: red',
        "DR": 'color: red; background-color: #FFB6C1',
        "MR": 'color: red',
        "DI": 'color: green; background-color: #F0FFF0',
        "MI": 'color: green',
        "HV": 'color: orange; background-color: #FAFAD2',
        "NS": 'color: gray',
        "NC": 'color: gray',
        "ID": 'color: red',
        fillna: 'color: #D3D3D3'
    }
    return '{}'.format(color_dict.get(val, 'color: black'))


def displayComparison(df):
    # These are the columns which need special formatting
    deltacols = df.columns.map(lambda x: x.endswith(("-CON", "Conclusion")))
    display(df.style
            .applymap(benchmark_color_delta, subset=deltacols)
            .applymap(bold_font, subset=deltacols)
            .format({'Test': lambda x: '<a target="_blank" href="{}">link</a>'.format(x), 'Base': lambda x: '<a target="_blank" href="{}">link</a>'.format(x)}))


In [ ]:
def get_fio_report_link(row, metadata):
#     info = {
#         "flask_addr": FLASK_ADDR,
#         "testrun": metadata.get('testrun-id'),
#         "platform": metadata.get('testrun-platform'),
#         "backend": metadata.get('hardware-disk-backend', ''),
#         'driver': metadata.get('hardware-disk-driver', ''),
#         'format': metadata.get('hardware-disk-format', ''),
# #        'iodepth': row['IOdepth'],
# #        'numjobs': row['Numjobs'],
# #        'bs': row['BS'],
# #        'rw': row['RW'],
#         'case_id': row['CaseID'],
#             }
#     url = "{}/storageresultpubview/list/?".format(FLASK_ADDR)
#     return "{flask_addr}/storageresultpubview/list/?\
# _flt_3_testrun={testrun_id}&\
# _flt_3_platform={platform}&\
# _flt_3_backend={backend}&\
# _flt_3_driver={driver}&\
# _flt_3_format={format}&\
# _flt_3_case_id={case_id}\
# ".format(**info)

    info = {
        "testrun": metadata.get('testrun-id'),
        "platform": metadata.get('testrun-platform'),
        "backend": metadata.get('hardware-disk-backend'),
        'driver': metadata.get('hardware-disk-driver'),
        'format': metadata.get('hardware-disk-format'),
        'case_id': row['CaseID'],
    }

    url = "{}/storageresultpubview/list/?".format(FLASK_ADDR)
    for key, value in info.items():
        if value is not None:
            url += '_flt_3_' + key + '=' + value + '&'
    return url


def get_uperf_report_link(row, metadata):
#     info = {
#         "flask_addr": FLASK_ADDR,
#         "testrun_id": metadata.get('testrun-id'),
#         "platform": metadata.get('testrun-platform'),
#         ''
#         'protocol': row['Protocol'],
#         'testtype': row['TestType'],
#         'msize': row['MSize'],
#         'instance': row['Instance']
#     }
#     return "{flask_addr}/networkresultpubview/list/?\
# _flt_0_testrun={testrun_id}&\
# _flt_0_platform={platform}&\
# _flt_0_protocol={protocol}&\
# _flt_0_testtype={testtype}&\
# _flt_0_msize={msize}&\
# _flt_0_instance={instance}\
# ".format(**info)

    ## flt_3 is "equal to", flt_0 is "starts with".

    info = {
        "testrun_id": metadata.get('testrun-id'),
        "platform": metadata.get('testrun-platform'),
        'case_id': row['CaseID'],
    }

    url = "{}/networkresultpubview/list/?".format(FLASK_ADDR)
    for key, value in info.items():
        if value is not None:
            url += '_flt_3_' + key + '=' + value + '&'
    return url

def get_report_link(row, metadata_file):
    with open(metadata_file) as f:
        metadata = json.load(f)
    testrun_type = metadata.get('testrun-type')
    if testrun_type == "uperf":
        return get_uperf_report_link(row, metadata)
    elif testrun_type == "fio":
        return get_fio_report_link(row, metadata)
    else:
        return FLASK_ADDR


benchmark_df = pd.read_csv(BENCHMARK, index_col=0, dtype=str, keep_default_na=False)
#summary_df = benchmark_df[['RW','BS','IOdepth','Numjobs']+list(benchmark_df.filter(regex='-CON$').columns)]
benchmark_df.insert(0, 'Test', benchmark_df.apply(lambda row: get_report_link(row, TEST_METADATA), axis=1))
benchmark_df.insert(0, 'Base', benchmark_df.apply(lambda row: get_report_link(row, BASE_METADATA), axis=1))

In [ ]:
import ipywidgets as widgets
import functools

# Get our unique values
ALL = 'ALL'
RESULT_COLUMN = "Conclusion"

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

filtered = pd.DataFrame()
output = widgets.Output()

# TBD: add more dropdown items
dropdown_list = []
# Result column dropdown widget
dropdown_list.append(widgets.Dropdown(
                                options = unique_sorted_values_plus_ALL(benchmark_df[RESULT_COLUMN]),
                                layout={'height':'30px', 'width':'20%'}))

# Details checkbox widget to show/hide -AVG, -%SD and SGN columns
detail_checkbox = widgets.Checkbox(
    value=False,
    description='Show Details',
    disabled=False,
    indent=True
)

# The detail checkbox description label is long, so need to customize label layout
detail_label = widgets.Label('Conclusion Filter:', layout=widgets.Layout(width='120px',height='30px'))

widget_list = [detail_label] + dropdown_list + [detail_checkbox]

def detail_filtering(detail, df):
    if not detail:
        show_cols = [col for col in df.columns.tolist() if ('-AVG' not in col and '-%SD' not in col and '-SGN' not in col)]
        return df[show_cols]
    else:
        return df

def common_filtering(cols, detail):
    """
    cols: dict, {'column_name': dropdown widget value}
    details: bool, checkbox widget value
    """
    
    global filtered
    output.clear_output()

    filters= []
    for name, value in cols.items():
        if value != ALL:
            filters.append(benchmark_df[name] == value)

    if filters:
        df_filter = functools.reduce(lambda x,y: x&y, filters)
        filtered = benchmark_df[df_filter]
    else:
        filtered = benchmark_df
        
    filtered = detail_filtering(detail, filtered)

    with output:
        displayComparison(filtered)
    
def result_dropdown_eventhandler(change):
    common_filtering({RESULT_COLUMN: change.new}, detail_checkbox.value)
    
def detail_checkbox_eventhandler(change):
    common_filtering({RESULT_COLUMN: dropdown_list[0].value}, change.new)

dropdown_list[0].observe(result_dropdown_eventhandler, names='value')
detail_checkbox.observe(detail_checkbox_eventhandler, names='value')

input_widgets = widgets.HBox(widget_list)
display(input_widgets)

# To workaround no horizon scrollbar issue
output.layout.width = '10000px'

with output:
    displayComparison(detail_filtering(False, benchmark_df))
display(output)